In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from keras.callbacks import LearningRateScheduler
from keras.layers import Dropout

In [2]:
# 图片路径
image_folder = "E:\shujuji\metamaterialpicture900"
# Excel文件路径
excel_path = "E:\\1-1000-90.xlsx"

In [3]:
df = pd.read_excel(excel_path, usecols=[0, 16])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行

In [4]:
# 初始化图像和性能数组
X = []
y = []

In [5]:
for i in range(1, 651):
    img_path = os.path.join(image_folder, f"lcy{i}.jpg")
    if i in df["编号"].values:
        img = load_img(img_path, target_size=(224, 224))  # 调整为合适的图像大小
        img_array = img_to_array(img)
        img_array = cv2.cvtColor(img_array.astype('uint8'), cv2.COLOR_RGB2GRAY)
        X.append(img_array)
y=df["d33"]

In [6]:
# 转换为NumPy数组
X = np.array(X)
y = np.array(y)

In [7]:
# 数据标准化
#scaler = StandardScaler()
#y = scaler.fit_transform(y.reshape(-1, 1)).flatten()

In [8]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)

In [9]:
from keras.layers import Dropout

# 构建卷积神经网络模型
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(224, 224, 1)))
model.add(AveragePooling2D((2, 2)))
model.add(Dropout(0.1))  # 调整 Dropout 的概率
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(AveragePooling2D((2, 2)))
model.add(Dropout(0.1))  # 调整 Dropout 的概率
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(AveragePooling2D((2, 2)))
model.add(Dropout(0.1))  # 调整 Dropout 的概率
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))  # 调整 Dropout 的概率
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))  # 调整 Dropout 的概率
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))  # 调整 Dropout 的概率
model.add(Dense(1, activation='linear'))

In [10]:
# 编译模型
model.compile(optimizer='adam', loss='mean_squared_error')

In [11]:
# 定义学习率调整函数
def lr_schedule(epoch):
    """
    学习率调整函数，可以根据需要调整学习率的方式来修改
    这里以每10个epoch将学习率减小为原来的0.9为例
    """
    initial_lr = 0.01
    drop = 0.9
    epochs_drop = 10
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr

# 创建一个 LearningRateScheduler 回调
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
# 在 model.fit 中使用 callbacks 参数将 LearningRateScheduler 加入训练过程
history = model.fit(X_train, y_train, epochs=40, batch_size=64, validation_data=(X_test, y_test), callbacks=[lr_scheduler])

Epoch 1/40
8/8 [==============================] - 20s 2s/step - loss: 6533649793024.0000 - val_loss: 40463.5117 - lr: 0.0100
Epoch 2/40
8/8 [==============================] - 19s 2s/step - loss: 49276.2812 - val_loss: 41081.8906 - lr: 0.0100
Epoch 3/40
8/8 [==============================] - 19s 2s/step - loss: 33114.1523 - val_loss: 19891.9277 - lr: 0.0100
Epoch 4/40
8/8 [==============================] - 19s 2s/step - loss: 23301.4980 - val_loss: 7737.8291 - lr: 0.0100
Epoch 5/40
8/8 [==============================] - 19s 2s/step - loss: 18432.4668 - val_loss: 7683.5146 - lr: 0.0100
Epoch 6/40
8/8 [==============================] - 19s 2s/step - loss: 18982.4707 - val_loss: 8711.7510 - lr: 0.0100
Epoch 7/40
8/8 [==============================] - 19s 2s/step - loss: 17349.2871 - val_loss: 9729.6445 - lr: 0.0100
Epoch 8/40
8/8 [==============================] - 19s 2s/step - loss: 14459.1670 - val_loss: 8828.0781 - lr: 0.0100
Epoch 9/40
8/8 [==============================] - 19s 2s/step

In [ ]:
plt.plot(history.history['loss'][5:], label='Training Loss')
plt.plot(history.history['val_loss'][5:], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 在训练集上计算 R² 分数
y_train_pred = model.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

# 在验证集上计算 R² 分数
y_test_pred = model.predict(X_test)
r2_test = r2_score(y_test, y_test_pred)

# 打印 R² 分数
print(f'R² 分数 - 训练集: {r2_train:.4f}')
print(f'R² 分数 - 验证集: {r2_test:.4f}')

In [ ]:
# 绘制真实值和预测值的对比图
plt.scatter(y_test, y_test_pred)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', label='Perfect Prediction')
plt.title('True vs Predicted Values')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()